In [50]:
%load_ext autoreload
%autoreload 2

from data.mouse import Mouse
from collections import defaultdict

import sys
sys.path.append('../')

from main import load_and_prepare_sessions
sessions = load_and_prepare_sessions("../../Baseline", first_n_dirs=2)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|██████████| 2/2 [00:05<00:00,  2.51s/it]


In [51]:
mice_sessions = defaultdict(list)
for session in sessions:
    mice_sessions[session.mouse_id].append(session)

In [52]:
mice = {mouse_id: Mouse(mouse_sessions) for mouse_id, mouse_sessions in mice_sessions.items()}

In [53]:
from scipy.stats import norm


def calculate_d_prime(hit_rate, false_alarm_rate):
    # Avoid division by zero or calculating z-scores for rates of 1 by capping the rates
    hit_rate = min(max(hit_rate, 1e-5), 1 - 1e-5)
    false_alarm_rate = min(max(false_alarm_rate, 1e-5), 1 - 1e-5)

    # Calculate z-scores
    z_hit = norm.ppf(hit_rate)
    z_false_alarm = norm.ppf(false_alarm_rate)

    # Calculate d'
    d_prime = z_hit - z_false_alarm

    return d_prime
mouse_d_primes = []

In [54]:
# Calculate d' for each mouse
mouse_d_primes = [(mouse_id, calculate_d_prime(mouse.rates['hit_rate'], mouse.rates['false_alarm_rate']))
                  for mouse_id, mouse in mice.items()]
mouse_d_primes = sorted(mouse_d_primes, key=lambda t: t[1])

# Display the results
for mouse_id, d_prime in mouse_d_primes:
    print(mouse_id, d_prime)

23 0.7802476264984495
29 0.9904741450481837
25 1.5260353101330202


In [55]:
sessions[0].timepoints_container.fetch_all_data_names()

['miss',
 'cor_reject',
 'hit',
 'reward_collect',
 'mistake',
 'before_dispimg_miss',
 'before_dispimg_cor_reject',
 'before_dispimg_hit',
 'before_dispimg_mistake']